In [1]:
import pandas as pd
from datetime import datetime

rawpath = '../raw'

In [2]:
# excel nach csv konvertieren
df = pd.read_excel(f'{rawpath}/Plakate_1918-20_ImportCBS.xlsx', names=['entstehungszeit','erscheinungsort','titel','verlagsort','sammlung41','sammlung67','signatur'])
df

,entstehungszeit,erscheinungsort,titel,verlagsort,sammlung41,sammlung67,signatur
0,1918,XA-DXDE,Nov.1.1,Deutschland,!1129808122!,!1132619645!,Nov.1.1
1,1918,XA-DXDE,Nov.1.3,Deutschland,!1129808122!,!1132619645!,Nov.1.3
2,1918,XA-DXDE,Nov.1.4,Deutschland,!1129808122!,!1132619645!,Nov.1.4
3,1920,XA-DXDE,Nov.10.10,NaN,!1129810429!,!1132743753!,Nov.10.10
4,1920,XA-DXDE,Nov.10.12,NaN,!1129810429!,!1132743753!,Nov.10.12
...,...,...,...,...,...,...,...
3783,1918-1920,XA-DXDE,Nov.9.2.92,NaN,!112987897X!,!1132743745!,Nov.9.2.92
3784,1918-1920,XA-DXDE,Nov.9.2.93,NaN,!112987897X!,!1132743745!,Nov.9.2.93
3785,1918-1920,XA-DXDE,Nov.9.2.94,NaN,!112987897X!,!1132743745!,Nov.9.2.94
3786,1918-1920,XA-DXDE,Nov.9.2.95,NaN,!112987897X!,!1132743745!,Nov.9.2.95


In [3]:
def pica_schreiben(row):
    pica = f"""\t\n0500 Pa
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy\n"""
    
    if len(str(row.entstehungszeit)) > 0 and len(str(row.entstehungszeit)) < 5:
        # nur ein datum
        pica += f"1100 {row.entstehungszeit}\n1110 *{row.entstehungszeit}$4ezth\n"
    elif len(str(row.entstehungszeit)) > 4 and len(str(row.entstehungszeit)) < 10:
        # 1100 1918$n[zwischen 1918 und 1920]
        # 1110 zwischen 1918 und 1920*1918-1920$a1918$b1920$4ezth
        pica += f"1100 {row.entstehungszeit[:4]}$n[zwischen {row.entstehungszeit[:4]} und {row.entstehungszeit[5:]}]\n1110 zwischen {row.entstehungszeit[:4]} und {row.entstehungszeit[5:]}*{row.entstehungszeit}$a{row.entstehungszeit[:4]}$b{row.entstehungszeit[5:]}$4ezth\n"
        # wohl zwei daten
    else:
        pica += "\n\n1100 FEHLER\n\n"
    
    pica += f"""1130 TB-papier
1130 !040445224!
1132 a1-analog;a2-druck;f2-blatt
1500 /1ger
1700 /1{row.erscheinungsort}
2105 04,P01
4000 Plakat Nr. {row.titel} der Revolutionssammlung 1918-1920
4019 Plakat Nr. {row.titel} der Revolutionssammlung 1918-1920\n"""
    
    if pd.isna(row.verlagsort):
        pica += "4030 [Ort nicht ermittelbar] : [Verlag nicht ermittelbar]\n"
    else:
        pica += f"4030 [{row.verlagsort.strip()}] : [Verlag nicht ermittelbar]\n"
    
    pica += f"""4060 1 ungezähltes Blatt
4105 !1122570686!
4105 {row.sammlung41}
4700 |BSM|
5590 [Objektgattung]
5590 !041133579!
7001 {now.strftime('%d-%m-%y')} : x
6710 {row.sammlung67.strip()}
6800 [Provenienz]
6800 !004709608! *1961-1993
6800 Stempel
7100 GS PL Nov @ m
7109 !!DBSM/S!! ; GS PL {row.titel}
8034 Dieses Plakat ist Teil eines Erschließungsprojekts und derzeit nicht ausleihbar.\n"""
    return pica


In [4]:
now = datetime.now()
with open(f"../dat/revolutionsplakate-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/revolutionsplakate-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/revolutionsplakate-recent-sample.dat", 'w') as f:
    for row in df.sample(5).itertuples():
        f.write(pica_schreiben(row))

In [4]:
now = datetime.now()
with open(f"../dat/revolutionsplakate-final-appr.dat", 'w') as f:
    for row in df.sample(378).itertuples():
        f.write(pica_schreiben(row))
